# Supervised ANNs for Mutual Information Investigation
## Thomas Possidente

### Imports and Initializations

In [1]:
# ANN Building and Visualization Imports
import keras
from keras import backend as K
from keras import optimizers, losses
from keras.engine.topology import Layer
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
import tensorflow as tf
from keras.callbacks import LambdaCallback
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import csv



# Standard Imports
import pandas as pd
import numpy as np
import math

# Value Inits - Specify as needed
num_inputs = int(5000)  # number of dummy images in set
size = int(16)          # Dimension of each dummy image should be size*size
RF_size = int(8)        # Dimensions of the RF to be analyzed should be RF_size*RF_size
noise = 0.5             # percentage (as decimal) of input values that will be flipped 

# Value Inits - Leave these alone
num_of_RFs = int((size*size) / (RF_size*RF_size))


C:\Users\Tom\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Reading in Data

In [2]:
inputs = pd.read_csv('Pattern_Matrices_Datasets/size16_RF8_noise0.csv')
inputs = inputs.drop('X1', axis = 0) # Taking out col names
inputs = inputs.apply(pd.to_numeric)  # converting to floats

inputs = inputs.values # convert to np ndarray
inputs = inputs.reshape(num_inputs, size, size,1) # reshape to desired dims (5000 examples, 16*16 image, 1 channel)
flattened_inputs = inputs.reshape(num_inputs, size*size)

### Cutting Number of Labels Possible to 256 if Necessary

In [ ]:
flattened_inputs = flattened_inputs[0:256]
flattened_inputs = np.repeat(flattened_inputs, 20, 0)
flattened_inputs = np.random.permutation(flattened_inputs)

num_inputs = int(flattened_inputs.shape[0])


### Creating Labels

In [3]:
labels = np.zeros(np.shape(flattened_inputs)[0])
storage = np.zeros(np.shape(flattened_inputs))
count = 0

for i in range(np.shape(flattened_inputs)[0]):
    if(~((flattened_inputs[i] == storage).all(1).any())):
        labels[i] = count
        storage[i] = flattened_inputs[i]
        count += 1
    elif((flattened_inputs[i] == storage).all(1).any()):
        label_index = np.where((flattened_inputs[i] == storage).all(1))
        correct_label = labels[label_index]
        storage[i] = flattened_inputs[i]
        labels[i] = correct_label[0]
        
labels = labels.astype(int)
labels = to_categorical(labels)

    

In [4]:
labels.shape

(5000, 256)

### Adding Noise to Inputs

In [5]:
for n in range(np.shape(flattened_inputs)[0]):
    indices_to_flip = np.random.choice(int(size*size), math.ceil(size*size * noise), replace = False)
    flattened_inputs[n][indices_to_flip] = 1 - flattened_inputs[n][indices_to_flip]

inputs = flattened_inputs.reshape(num_inputs, size, size, 1)


In [ ]:
model = Sequential()
model.add(Conv2D(input_shape = (16,16,1), filters=4, kernel_size = 8, strides = 8, activation = 'relu'))
model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(Dense(256, activation = 'softmax'))
model.summary()

### Building Network

In [6]:
def run_model():
    
    model = Sequential()
    model.add(Conv2D(input_shape = (16,16,1), filters=4, kernel_size = RF_size, strides = RF_size, activation = 'relu'))
    model.add(Flatten())
    model.add(Dense(512, activation = 'relu'))
    model.add(Dense(256, activation = 'softmax'))
    
    model.compile(optimizer = optimizers.adam(lr = 0.0005), loss = 'categorical_crossentropy', metrics=['accuracy'])
    
    history = model.fit(x = inputs, validation_split = 0.25, y = labels, batch_size = 100, epochs = 25, shuffle = True)
    
    return(history)



In [99]:
storage_raw_acc = np.zeros((30,27))
storage_raw_loss = np.zeros((30,27))

storage_means_acc = np.zeros((3, 27))
storage_means_loss = np.zeros((3, 27))
storage_means_acc_SDs = np.zeros((3, 27))
storage_means_loss_SDs = np.zeros((3, 27))


param_RF_size = np.array([2,4,8])
num_inputs = int(5120) 
size = int(16)
noise = 50
counter = 0

for n in range(3):
    RF_size = int(param_RF_size[n])
    
    val_acc = np.zeros((10,25))
    val_loss = np.zeros((10,25))
    
    for i in range(10):
        results = run_model()
        val_acc[i] = results.history['val_acc']
        val_loss[i] = results.history['val_loss']
        storage_raw_acc[counter] = np.hstack((RF_size, noise, val_acc[i]))
        storage_raw_loss[counter] = np.hstack((RF_size, noise, val_loss[i]))
        counter += 1

    acc_means = np.mean(val_acc, axis = 0)
    loss_means = np.mean(val_loss, axis = 0)
    acc_SD = np.std(val_acc, axis = 0)
    loss_SD = np.std(val_loss, axis = 0)

    
    storage_means_acc[n] = np.hstack((RF_size, noise, acc_means))
    storage_means_acc_SDs[n] = np.hstack((RF_size, noise, acc_SD))
    storage_means_loss[n] = np.hstack((RF_size, noise, loss_means))
    storage_means_loss_SDs[n] = np.hstack((RF_size, noise, loss_SD))

    
    
#print("acc15 = " + str(np.mean(val_acc15)), "\n",
#      "acc100 = " + str(np.mean(val_acc99)), "\n",
#      "loss15 = " + str(np.mean(val_loss15)), "\n",
#      "loss100 = " + str(np.mean(val_loss99)), "\n",
#      "acc15_SD = " + str(np.std(val_acc15)), "\n",
#      "acc100_SD = " + str(np.std(val_acc99)), "\n",
#      "loss15_SD = " + str(np.std(val_loss15)), "\n",
#      "loss100_SD = " + str(np.std(val_loss99)))

Train on 3750 samples, validate on 1250 samples
Epoch 1/25
3750/3750 [==============================] - 17s 4ms/step - loss: 5.5650 - acc: 0.0067 - val_loss: 5.5622 - val_acc: 0.0040
Epoch 2/25
3750/3750 [==============================] - 3s 729us/step - loss: 5.4771 - acc: 0.0085 - val_loss: 5.5805 - val_acc: 0.0024
Epoch 3/25
3750/3750 [==============================] - 3s 729us/step - loss: 5.4085 - acc: 0.0131 - val_loss: 5.5989 - val_acc: 0.0024
Epoch 4/25
3750/3750 [==============================] - 3s 716us/step - loss: 5.3101 - acc: 0.0189 - val_loss: 5.6210 - val_acc: 0.0032
Epoch 5/25
3750/3750 [==============================] - 3s 736us/step - loss: 5.1902 - acc: 0.0496 - val_loss: 5.6262 - val_acc: 0.0024
Epoch 6/25
3750/3750 [==============================] - 3s 726us/step - loss: 5.0392 - acc: 0.0643 - val_loss: 5.6692 - val_acc: 0.0056
Epoch 7/25
3750/3750 [==============================] - 3s 730us/step - loss: 4.8614 - acc: 0.1235 - val_loss: 5.7029 - val_acc: 0.0048
E

Epoch 10/25
3750/3750 [==============================] - 3s 723us/step - loss: 4.3284 - acc: 0.2272 - val_loss: 5.8855 - val_acc: 0.0032
Epoch 11/25
3750/3750 [==============================] - 3s 761us/step - loss: 4.0953 - acc: 0.2733 - val_loss: 5.9854 - val_acc: 0.0016
Epoch 12/25
3750/3750 [==============================] - 3s 743us/step - loss: 3.8553 - acc: 0.3104 - val_loss: 6.0941 - val_acc: 0.0040
Epoch 13/25
3750/3750 [==============================] - 3s 739us/step - loss: 3.6103 - acc: 0.3501 - val_loss: 6.2235 - val_acc: 0.0016
Epoch 14/25
3750/3750 [==============================] - 3s 729us/step - loss: 3.3526 - acc: 0.3947 - val_loss: 6.3172 - val_acc: 0.0000e+00
Epoch 15/25
3750/3750 [==============================] - 3s 739us/step - loss: 3.1183 - acc: 0.4264 - val_loss: 6.4331 - val_acc: 0.0040
Epoch 16/25
3750/3750 [==============================] - 3s 753us/step - loss: 2.8942 - acc: 0.4739 - val_loss: 6.6210 - val_acc: 0.0040
Epoch 17/25
3750/3750 [==============

3750/3750 [==============================] - 3s 754us/step - loss: 1.7961 - acc: 0.6920 - val_loss: 7.4048 - val_acc: 0.0024
Epoch 20/25
3750/3750 [==============================] - 3s 737us/step - loss: 1.6173 - acc: 0.7341 - val_loss: 7.5958 - val_acc: 0.0024
Epoch 21/25
3750/3750 [==============================] - 3s 752us/step - loss: 1.4647 - acc: 0.7608 - val_loss: 7.7432 - val_acc: 0.0040
Epoch 22/25
3750/3750 [==============================] - 3s 757us/step - loss: 1.3076 - acc: 0.7989 - val_loss: 7.9013 - val_acc: 8.0000e-04
Epoch 23/25
3750/3750 [==============================] - 3s 767us/step - loss: 1.1733 - acc: 0.8293 - val_loss: 8.0528 - val_acc: 0.0016
Epoch 24/25
3750/3750 [==============================] - 3s 758us/step - loss: 1.0492 - acc: 0.8525 - val_loss: 8.1868 - val_acc: 0.0024
Epoch 25/25
3750/3750 [==============================] - 3s 743us/step - loss: 0.9254 - acc: 0.8837 - val_loss: 8.4192 - val_acc: 0.0024
Train on 3750 samples, validate on 1250 samples
E

Epoch 3/25
3750/3750 [==============================] - 3s 749us/step - loss: 5.4264 - acc: 0.0101 - val_loss: 5.5944 - val_acc: 0.0024
Epoch 4/25
3750/3750 [==============================] - 3s 754us/step - loss: 5.3382 - acc: 0.0192 - val_loss: 5.6154 - val_acc: 8.0000e-04
Epoch 5/25
3750/3750 [==============================] - 3s 757us/step - loss: 5.2224 - acc: 0.0419 - val_loss: 5.6386 - val_acc: 8.0000e-04
Epoch 6/25
3750/3750 [==============================] - 3s 758us/step - loss: 5.0698 - acc: 0.0669 - val_loss: 5.6591 - val_acc: 0.0016
Epoch 7/25
3750/3750 [==============================] - 3s 744us/step - loss: 4.8777 - acc: 0.1141 - val_loss: 5.6841 - val_acc: 0.0048
Epoch 8/25
3750/3750 [==============================] - 3s 758us/step - loss: 4.6456 - acc: 0.1739 - val_loss: 5.7328 - val_acc: 0.0040
Epoch 9/25
3750/3750 [==============================] - 3s 747us/step - loss: 4.3809 - acc: 0.2499 - val_loss: 5.8342 - val_acc: 0.0032
Epoch 10/25
3750/3750 [=================

3750/3750 [==============================] - 3s 695us/step - loss: 3.4988 - acc: 0.2611 - val_loss: 6.8715 - val_acc: 8.0000e-04
Epoch 22/25
3750/3750 [==============================] - 3s 700us/step - loss: 3.4184 - acc: 0.2795 - val_loss: 6.9701 - val_acc: 0.0032
Epoch 23/25
3750/3750 [==============================] - 3s 688us/step - loss: 3.3279 - acc: 0.2901 - val_loss: 7.0978 - val_acc: 0.0032
Epoch 24/25
3750/3750 [==============================] - 3s 697us/step - loss: 3.2568 - acc: 0.3000 - val_loss: 7.1713 - val_acc: 8.0000e-04
Epoch 25/25
3750/3750 [==============================] - 3s 698us/step - loss: 3.1786 - acc: 0.3107 - val_loss: 7.2602 - val_acc: 8.0000e-04
Train on 3750 samples, validate on 1250 samples
Epoch 1/25
3750/3750 [==============================] - 17s 5ms/step - loss: 5.5495 - acc: 0.0043 - val_loss: 5.5578 - val_acc: 0.0032
Epoch 2/25
3750/3750 [==============================] - 3s 699us/step - loss: 5.4942 - acc: 0.0112 - val_loss: 5.5708 - val_acc: 0.0

3750/3750 [==============================] - 2s 636us/step - loss: 4.2579 - acc: 0.1683 - val_loss: 6.1800 - val_acc: 0.0016
Epoch 14/25
3750/3750 [==============================] - 2s 645us/step - loss: 4.1334 - acc: 0.1808 - val_loss: 6.2859 - val_acc: 0.0064
Epoch 15/25
3750/3750 [==============================] - 2s 634us/step - loss: 4.0118 - acc: 0.1920 - val_loss: 6.4104 - val_acc: 0.0032
Epoch 16/25
3750/3750 [==============================] - 2s 647us/step - loss: 3.8921 - acc: 0.2101 - val_loss: 6.5389 - val_acc: 0.0032
Epoch 17/25
3750/3750 [==============================] - 2s 636us/step - loss: 3.7836 - acc: 0.2275 - val_loss: 6.6222 - val_acc: 0.0048
Epoch 18/25
3750/3750 [==============================] - 2s 653us/step - loss: 3.6884 - acc: 0.2363 - val_loss: 6.7371 - val_acc: 0.0032
Epoch 19/25
3750/3750 [==============================] - 2s 640us/step - loss: 3.5984 - acc: 0.2547 - val_loss: 6.8613 - val_acc: 0.0040
Epoch 20/25
3750/3750 [==============================

3750/3750 [==============================] - 2s 559us/step - loss: 5.3855 - acc: 0.0131 - val_loss: 5.6107 - val_acc: 0.0016
Epoch 7/25
3750/3750 [==============================] - 2s 556us/step - loss: 5.3438 - acc: 0.0165 - val_loss: 5.6281 - val_acc: 0.0016
Epoch 8/25
3750/3750 [==============================] - 2s 554us/step - loss: 5.2965 - acc: 0.0211 - val_loss: 5.6423 - val_acc: 0.0024
Epoch 9/25
3750/3750 [==============================] - 2s 550us/step - loss: 5.2469 - acc: 0.0240 - val_loss: 5.6616 - val_acc: 0.0016
Epoch 10/25
3750/3750 [==============================] - 2s 566us/step - loss: 5.1972 - acc: 0.0288 - val_loss: 5.6874 - val_acc: 0.0016
Epoch 11/25
3750/3750 [==============================] - 2s 565us/step - loss: 5.1520 - acc: 0.0293 - val_loss: 5.7143 - val_acc: 0.0016
Epoch 12/25
3750/3750 [==============================] - 2s 549us/step - loss: 5.1069 - acc: 0.0365 - val_loss: 5.7438 - val_acc: 0.0032
Epoch 13/25
3750/3750 [==============================] -

3750/3750 [==============================] - 2s 548us/step - loss: 4.7092 - acc: 0.0632 - val_loss: 6.1356 - val_acc: 0.0048
Epoch 25/25
3750/3750 [==============================] - 2s 550us/step - loss: 4.6848 - acc: 0.0699 - val_loss: 6.1677 - val_acc: 0.0040
Train on 3750 samples, validate on 1250 samples
Epoch 1/25
3750/3750 [==============================] - 16s 4ms/step - loss: 5.5486 - acc: 0.0048 - val_loss: 5.5447 - val_acc: 0.0032
Epoch 2/25
3750/3750 [==============================] - 2s 537us/step - loss: 5.5218 - acc: 0.0104 - val_loss: 5.5521 - val_acc: 0.0016
Epoch 3/25
3750/3750 [==============================] - 2s 615us/step - loss: 5.4900 - acc: 0.0085 - val_loss: 5.5781 - val_acc: 0.0024
Epoch 4/25
3750/3750 [==============================] - 2s 541us/step - loss: 5.4532 - acc: 0.0085 - val_loss: 5.5887 - val_acc: 0.0032
Epoch 5/25
3750/3750 [==============================] - 2s 553us/step - loss: 5.4166 - acc: 0.0109 - val_loss: 5.6141 - val_acc: 0.0040
Epoch 6/25


Epoch 8/25
3750/3750 [==============================] - 2s 546us/step - loss: 5.3075 - acc: 0.0176 - val_loss: 5.6525 - val_acc: 0.0016
Epoch 9/25
3750/3750 [==============================] - 2s 534us/step - loss: 5.2602 - acc: 0.0240 - val_loss: 5.6894 - val_acc: 0.0024
Epoch 10/25
3750/3750 [==============================] - 2s 540us/step - loss: 5.2133 - acc: 0.0293 - val_loss: 5.7075 - val_acc: 8.0000e-04
Epoch 11/25
3750/3750 [==============================] - 2s 535us/step - loss: 5.1675 - acc: 0.0293 - val_loss: 5.7363 - val_acc: 0.0024
Epoch 12/25
3750/3750 [==============================] - 2s 548us/step - loss: 5.1220 - acc: 0.0365 - val_loss: 5.7648 - val_acc: 0.0016
Epoch 13/25
3750/3750 [==============================] - 2s 532us/step - loss: 5.0799 - acc: 0.0376 - val_loss: 5.8024 - val_acc: 0.0016
Epoch 14/25
3750/3750 [==============================] - 2s 537us/step - loss: 5.0408 - acc: 0.0384 - val_loss: 5.8451 - val_acc: 8.0000e-04
Epoch 15/25
3750/3750 [============

In [100]:
storage_raw_acc1 = pd.DataFrame(storage_raw_acc)
storage_raw_loss1 = pd.DataFrame(storage_raw_loss)

storage_means_acc1 = pd.DataFrame(storage_means_acc)
storage_means_loss1 = pd.DataFrame(storage_means_loss)
storage_means_acc_SDs1 = pd.DataFrame(storage_means_acc_SDs)
storage_means_loss_SDs1 = pd.DataFrame(storage_means_loss_SDs)

In [101]:
# appending to a csv #

with open('storage_raw_acc1.csv','a') as csv_file:
    writer = csv.writer(csv_file, lineterminator = '\n')
    for i in range(storage_raw_acc1.shape[0]):
        writer.writerow(storage_raw_acc1.iloc[i,])
csv_file.close()
    
    
with open('storage_raw_loss1.csv','a') as csv_file:
    writer = csv.writer(csv_file, lineterminator = '\n')
    for i in range(storage_raw_loss1.shape[0]):
        writer.writerow(storage_raw_loss1.iloc[i,])
csv_file.close()


with open('storage_means_acc1.csv','a') as csv_file:
    writer = csv.writer(csv_file, lineterminator = '\n')
    for i in range(storage_means_acc1.shape[0]):
        writer.writerow(storage_means_acc1.iloc[i,])
csv_file.close()


with open('storage_means_loss1.csv','a') as csv_file:
    writer = csv.writer(csv_file, lineterminator = '\n')
    for i in range(storage_means_loss1.shape[0]):
        writer.writerow(storage_means_loss1.iloc[i,])
csv_file.close()

with open('storage_SDs_acc1.csv','a') as csv_file:
    writer = csv.writer(csv_file, lineterminator = '\n')
    for i in range(storage_means_acc_SDs1.shape[0]):
        writer.writerow(storage_means_acc_SDs1.iloc[i,])
csv_file.close()
    
    
with open('storage_SDs_loss1.csv','a') as csv_file:
    writer = csv.writer(csv_file, lineterminator = '\n')
    for i in range(storage_means_loss_SDs1.shape[0]):
        writer.writerow(storage_means_loss_SDs1.iloc[i,])
csv_file.close()
   

In [ ]:
# Writing first csv #

# storage_raw_acc1.to_csv('storage_raw_acc1.csv')
# storage_raw_loss1.to_csv('storage_raw_loss1.csv')

# storage_means_acc1.to_csv('storage_means_acc1.csv')
# storage_means_loss1.to_csv('storage_means_loss1.csv')
# storage_means_acc_SDs1.to_csv('storage_SDs_acc1.csv')
# storage_means_loss_SDs1.to_csv('storage_SDs_loss1.csv')

### TODOS
* Find Image dataset that's binarized (or binarize existing one) and do MI calculations to find max MI. To do this, sample RF sized chunks of an image until stable average MI value is reached
* https://people.cs.umass.edu/~marlin/data.shtml, http://vision.lems.brown.edu/content/available-software-and-databases